In [ ]:
import tweepy
import pandas as pd
import pyodbc

# Replace these with your own credentials
bearer_token = '<Paste the bearer token here>'

# Authenticate to Twitter
client = tweepy.Client(bearer_token=bearer_token)

# Define the search query and date range
query = 'Edo Election -is:retweet'  # Replace with your search term

# Can not go beyond 7 days
start_time = '2024-09-20T00:30:00Z'  # Replace with your desired start date
end_time = '2024-09-21T23:59:59Z'    # Replace with your desired end date

# Collect tweets
tweets = client.search_recent_tweets(query=query, tweet_fields=['author_id', 'created_at'], max_results=100, start_time=start_time, end_time=end_time)

# Check if tweets were returned
if tweets.data:
    # Store tweets in a dataframe
    tweets_data = [[tweet.author_id, tweet.text, tweet.created_at, query, start_time, end_time] for tweet in tweets.data]
    df = pd.DataFrame(data=tweets_data, columns=['User', 'Tweet', 'Date', 'Keyword', 'Start_Date', 'End_Date'])

    # Connect to SQL Server using Windows Authentication
    conn = pyodbc.connect('DRIVER={SQL Server};'
                          'SERVER=Adediwura;'
                          'DATABASE=Thesis;'
                          'Trusted_Connection=yes;')

    cursor = conn.cursor()

    # Insert data into the table
    for index, row in df.iterrows():
        cursor.execute("INSERT INTO Tweets ([User], [Tweet], [Date], [Keyword], [Start_Date], [End_Date]) VALUES (?, ?, ?, ?, ?, ?)", 
                       row['User'], row['Tweet'], row['Date'], row['Keyword'], row['Start_Date'], row['End_Date'])

    # Commit and close the connection
    conn.commit()
    cursor.close()
    conn.close()

    print("Tweets inserted into the database successfully.")
else:
    print("No tweets found for the specified query and date range.")